In [72]:
class MathParser:
    def __init__(self, expression):
        self.expression = expression.replace(' ', '')
        self.index = 0

    def parse(self):
        return self.parse_expression()

    def parse_expression(self):
        values = [self.parse_term()]
        while self.peek() in '+-':
            if self.consume('+'):
                values.append(self.parse_term())
            elif self.consume('-'):
                values.append(-self.parse_term())
        return sum(values)

    def parse_term(self):
        values = [self.parse_factor()]
        while self.peek() in '*/':
            if self.consume('*'):
                values.append(self.parse_factor())
            elif self.consume('/'):
                divisor = self.parse_factor()
                if divisor != 0:
                    values[-1] /= divisor
                else:
                    raise ValueError("Division by zero")
        product = 1
        for v in values:
            product *= v
        return product

    def parse_factor(self):
        if self.peek() == '(':
            self.consume('(')
            value = self.parse_expression()
            self.expect(')')
            return value
        else:
            return self.parse_number()

    def parse_number(self):
        num = ''
        while self.peek().isdigit() or self.peek() == '.':
            num += self.consume()
        try:
            return float(num)
        except ValueError:
            raise ValueError(f"Invalid number: {num}")

    def peek(self):
        return self.expression[self.index] if self.index < len(self.expression) else ''

    def consume(self, char=None):
        if char and self.expression[self.index] != char:
            raise ValueError(f"Expected '{char}' but found '{self.expression[self.index]}'")
        result = self.expression[self.index]
        self.index += 1
        return result

    def expect(self, char):
        if not self.consume(char):
            raise ValueError(f"Expected '{char}'")


def simplify_equation(equation):
    # Create a MathParser instance
    parser = MathParser(equation)

    # Parse and evaluate the expression
    result = parser.parse()

    return result

# Example usage
equation = "x**2 + 4 - 5 + x + x*x + 12 - 3*(4/4213)"
simplified_equation = simplify_equation(equation)
print(simplified_equation)

ValueError: Invalid number: 

In [92]:
def simplify_equation(equation):
    # Handle multiplication and prepare for splitting
    equation = equation.replace('x*x', 'x**2').replace('-', '+-')
    parts = equation.split('+')
    
    # Initialize a dictionary for coefficients
    coefficients = {}
    
    # Process each part
    for part in parts:
        part = part.strip()
        if not part:  # Skip empty parts
            continue
        
        # Determine the power of x
        if 'x**' in part:
            power = int(part.split('x**')[1])
            coef_part = part.split('x**')[0]
        elif 'x' in part:
            power = 1
            coef_part = part.split('x')[0]
        else:
            power = 0
            coef_part = part
        
        coef_part = coef_part.replace('*', '') 
        if coef_part in ('', '+'):
            coefficient = 1
        elif coef_part == '-':
            coefficient = -1
        else:
            try:
                coefficient = int(coef_part)
            except ValueError:
                continue
        
        # Aggregate coefficients
        coefficients[power] = coefficients.get(power, 0) + coefficient
    
    # Construct simplified equation
    simplified_parts = []
    for power in sorted(coefficients, reverse=True):
        coefficient = coefficients[power]
        if coefficient == 0:
            continue  # Skip zero coefficients
        if power == 0:
            simplified_parts.append(str(coefficient))
        elif power == 1:
            if coefficient == 1:
                simplified_parts.append("x")
            elif coefficient == -1:
                simplified_parts.append("-x")
            else:
                simplified_parts.append(f"{coefficient}*x")
        else:
            if coefficient == 1:
                simplified_parts.append(f"x**{power}")
            elif coefficient == -1:
                simplified_parts.append(f"-x**{power}")
            else:
                simplified_parts.append(f"{coefficient}*x**{power}")
    
    simplified_equation = '+'.join(simplified_parts).replace('+-', '-')
    first_char, rest_expression = simplified_equation[0], simplified_equation[1:]
    spaced_expression = first_char + re.sub(r'([+\-])', r' \1 ', rest_expression)

    return spaced_expression

# Example usage
equation = "10*x**3 +4-5+x+x*x+12-3*(4/4213)-4x**3"
simplified_equation = simplify_equation(equation)

print(simplified_equation)


6*x**3 + x**2 + x + 11


In [88]:
# Your input expression
expression = "-4*x**3+3*x**2+x+11"

# Split the expression into the first character and the rest
first_char, rest_expression = expression[0], expression[1:]

# Add spaces around plus and minus signs in the rest of the expression
spaced_expression = first_char + re.sub(r'([+\-])', r' \1 ', rest_expression)

# Print the spaced expression
print(spaced_expression)


-4*x**3 + 3*x**2 + x + 11


In [34]:
def plot_function(f, xmin, xmax, width=80, height=20, x_range=10):
    def derivative(f, x, h=0.0001):
        return (f(x + h) - f(x - h)) / (2 * h)
    
    def find_turning_points(f, xmin, xmax, steps=1000):
        x_values = [xmin + i * (xmax - xmin) / steps for i in range(steps + 1)]
        turning_points = []

        for i in range(1, len(x_values)):
            x0, x1 = x_values[i - 1], x_values[i]
            dy_dx0, dy_dx1 = derivative(f, x0), derivative(f, x1)

            if dy_dx0 * dy_dx1 < 0 or dy_dx0 == 0:
                turning_points.append((x0 + x1) / 2)

        return turning_points

    turning_points_x = find_turning_points(f, xmin, xmax)
    if turning_points_x:
        center_x = sum(turning_points_x) / len(turning_points_x)
    else:
        center_x = (xmin + xmax) / 2

    xmin = center_x - x_range / 2
    xmax = center_x + x_range / 2

    ymin, ymax = min([f(x) for x in turning_points_x] + [f(xmin), f(xmax)]), max([f(x) for x in turning_points_x] + [f(xmin), f(xmax)])
    ypad = (ymax - ymin) * 0.1
    ymin, ymax = ymin - ypad, ymax + ypad

    plot = [[' ' for _ in range(width)] for _ in range(height)]

    for i in range(width):
        x = xmin + (xmax - xmin) * i / width
        y = f(x)
        if ymin <= y <= ymax:
            plot_y = int((y - ymin) / (ymax - ymin) * (height - 1))
            plot[height - plot_y - 1][i] = '*'

    for tp_x in turning_points_x:
        x_pos = int((tp_x - xmin) / (xmax - xmin) * width)
        y = f(tp_x)
        if ymin <= y <= ymax:
            plot_y = int((y - ymin) / (ymax - ymin) * (height - 1))
            plot[height - plot_y - 1][x_pos] = 'X'

    for row in plot:
        print(''.join(row))
    
    if turning_points_x:
        print("\nTurning Points Centered Plot:")
    for tp_x in turning_points_x:
        print(f"x ≈ {tp_x:.1f}, y ≈ {f(tp_x):.1f}")

def math_function(x):
    return x**3


plot_function(math_function, xmin=-10, xmax=10, width=80, height=20, x_range=10)


                                                                                
                                                                                
                                                                                
                                                                              **
                                                                            **  
                                                                          **    
                                                                       ***      
                                                                    ***         
                                                               *****            
                                                        *******                 
                         *******************************                        
                  *******                                                       
             *****          

In [36]:
def plot_function_with_turning_points(f, xmin, xmax, width=80, height=20):
    # Numerically estimate the derivative and find turning points
    def derivative(f, x, h=0.0001):
        return (f(x + h) - f(x - h)) / (2 * h)
    
    def find_turning_points(f, xmin, xmax, steps=1000):
        x_values = [xmin + i * (xmax - xmin) / steps for i in range(steps + 1)]
        turning_points = []

        for i in range(1, len(x_values)):
            x0, x1 = x_values[i - 1], x_values[i]
            dy_dx0, dy_dx1 = derivative(f, x0), derivative(f, x1)

            #check if the derivative changes sign
            if dy_dx0 * dy_dx1 < 0 or dy_dx0 == 0:
                turning_points.append((x0 + x1) / 2)  # approximate turning point at midpoint

        return turning_points

    turning_points_x = find_turning_points(f, xmin, xmax)
    turning_points_y = [f(x) for x in turning_points_x]

    # Set the plot range to include all turning points
    ymin, ymax = min(turning_points_y + [f(xmin), f(xmax)]), max(turning_points_y + [f(xmin), f(xmax)])
    ypad = (ymax - ymin) * 0.1
    ymin, ymax = ymin - ypad, ymax + ypad

    # ASCII plotting
    plot = [[' ' for _ in range(width)] for _ in range(height)]

    for i in range(width):
        x = xmin + (xmax - xmin) * i / width
        y = f(x)
        if ymin <= y <= ymax:
            plot_y = int((y - ymin) / (ymax - ymin) * (height - 1))
            plot[height - plot_y - 1][i] = '*'

    # Mark turning points on the plot
    for tp_x in turning_points_x:
        x_pos = int((tp_x - xmin) / (xmax - xmin) * width)
        y = f(tp_x)
        if ymin <= y <= ymax:
            plot_y = int((y - ymin) / (ymax - ymin) * (height - 1))
            plot[height - plot_y - 1][x_pos] = 'X'  # Mark turning points with 'X'

    # Print the plot
    for row in plot:
        print(''.join(row))
    
    # Print turning points info
    if turning_points_x:
        print("\nTurning Points:")
    for tp_x in turning_points_x:
        print(f"x ≈ {tp_x:.2f}, y ≈ {f(tp_x):.2f}")

# Example function
def my_function(x):
    return x**2

# Plotting the function and its turning points
plot_function_with_turning_points(my_function, xmin=-2, xmax=5, width=80, height=20)


                                                                                
                                                                                
                                                                                
                                                                              **
                                                                            **  
                                                                          **    
                                                                        **      
                                                                      **        
                                                                   ***          
                                                                 **             
                                                              ***               
                                                            **                  
                            

In [28]:
def plot_approx_turning_points(func, xmin, xmax, width=60, height=20):
    # find approximate turning points
    x_values = [xmin + (xmax - xmin) * i / (width - 1) for i in range(width)]
    y_values = [func(x) for x in x_values]

    # min and max y-values to set the plot range
    ymin, ymax = min(y_values), max(y_values)

    #  y-range to include all turning points
    y_range = ymax - ymin
    ymax += y_range * 0.1
    ymin -= y_range * 0.1
    
    # create the grid
    grid = [[' ' for _ in range(width)] for _ in range(height)]
    
    # plot the function
    for i, x in enumerate(x_values):
        real_y = y_values[i]
        if ymin <= real_y <= ymax:
            y = height - 1 - int((real_y - ymin) / (ymax - ymin) * (height - 1))
            grid[y][i] = '+'
    
    # Plot axes
    if xmin * xmax < 0:  # X-axis crosses the plot
        y_axis_pos = int(-xmin / (xmax - xmin) * (width - 1))
        for row in grid:
            row[y_axis_pos] = '|'
    if ymin * ymax < 0:  # Y-axis crosses the plot
        x_axis_pos = height - 1 - int(-ymin / (ymax - ymin) * (height - 1))
        for i in range(width):
            grid[x_axis_pos][i] = '-'
    
    # Print the grid
    for row in grid:
        print(''.join(row))

# Define a more complex function
def my_complex_func(x):
    return x**3+67

# Example usage
plot_approx_turning_points(my_complex_func, xmin=-10, xmax=10, width=80, height=30)


Turning points:
x ≈ 1.00, y ≈ 4.00
x ≈ 3.00, y ≈ 0.00


In [6]:
def plot_ascii_equation(func, xmin, xmax, ymin, ymax, width=40, height=20):
    # Create the grid
    grid = [[' ' for _ in range(width)] for _ in range(height)]
    
    # Calculate the step sizes
    xstep = (xmax - xmin) / (width - 1)
    ystep = (ymax - ymin) / (height - 1)
    
    # Plot the axes
    y_axis_pos = round(-xmin / xstep)
    x_axis_pos = round(-ymin / ystep)
    
    if 0 <= y_axis_pos < width:
        for i in range(height):
            grid[i][y_axis_pos] = '|'
    if 0 <= x_axis_pos < height:
        for i in range(width):
            grid[x_axis_pos][i] = '_'
    
    # Plot the function
    for x in range(width):
        real_x = xmin + x * xstep
        real_y = func(real_x)
        if ymin <= real_y <= ymax:
            y = height - 1 - round((real_y - ymin) / ystep)
            if 0 <= y < height:
                grid[y][x] = '+'
    
    # Print the grid
    for row in grid:
        print(''.join(row))

# Example usage
def my_func(x):
    return x*2+10

plot_ascii_equation(my_func, -10, 10, 0, 100, width=100, height=50)


____________________________________________________________________________________________________
                                                  |                                                 
                                                  |                                                 
                                                  |                                                 
                                                  |                                                 
                                                  |                                                 
                                                  |                                                 
                                                  |                                                 
                                                  |                                                 
                                                  |                                        

In [75]:
import re
import os

class DictionaryHandler:
    def add_to_dict(self, assignment_input, assignments):
        key, value = assignment_input.replace(" ", "").split("=")

        assignments[key] = value
        return assignments
        
class InputHandler:
    def __init__(self):
        self.assignment_pattern = re.compile(r'^[a-zA-Z0-9+\-*/**()= ]+$')

    def is_valid_assignment(self, assignment_string):
        if not assignment_string:
            print("Input Error: Assignment cannot be empty")
            return False
        elif not self.assignment_pattern.match(assignment_string):
            print("Input Error: Use of Invalid characters")
            return False
        elif '=' not in assignment_string:
            print("Input Error: Assignment must contain '='")
            return False
        else:
            return True
        
    def is_valid_filename(self, filename):
        if not filename:
            print("Filename cannot be empty")
            return False

        if not os.path.basename(filename) == filename:
            print("File cannot be fun")
            return False

        if '.' not in filename:
            filename += '.txt'

        return True
 

class EquationSorter:
    def __init__(self):
        self.eq_dict = {}
        self.dependency_graph = {}

    def _add_dependencies(self, var, sorted_eqs, seen, undefined_vars):
        if var in seen:
            raise ValueError(f"Circular dependency detected involving '{var}'")
        if var not in sorted_eqs:
            if var not in self.dependency_graph:
                undefined_vars.add(var)
                return
            seen.add(var)
            for dep in self.dependency_graph.get(var, []):
                self._add_dependencies(dep, sorted_eqs, seen, undefined_vars)
            sorted_eqs[var] = self.eq_dict[var]
            seen.remove(var)

    def sort_equations(self, equations):
        # parsing the equations to extract variables and their dependencies
        for eq in equations:
            var, expr = eq.split(" = ")
            deps = set([x for x in expr.split(" ") if x.isalpha() and x != var])
            self.eq_dict[var] = expr
            self.dependency_graph[var] = deps

        # sorting the equations based on dependencies
        sorted_eqs = {}
        seen = set()
        undefined_vars = set()
        for var in self.dependency_graph:
            self._add_dependencies(var, sorted_eqs, seen, undefined_vars)

        # adding undefined variables at the end
        for var in undefined_vars:
            sorted_eqs[var] = None

        return sorted_eqs

In [76]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        # define criteria for variable
        return char.isalpha()

    def _isoperator(self, char):
        # define criteria for operator
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif exp[i] == '-':
                if (i != 0 and (exp[i - 1]).isdigit() or self._isvariable(exp[i - 1]) or exp[i - 1] == ')') and ((exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]) or self._isoperator(exp[i + 1])):
                    tokens.append(exp[i])
                    i += 1
                else:
                    if (exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]):
                        tokens.extend(['(', exp[i + 1], '*', '-1', ')'])
                        i += 2
                    elif exp[i + 1] == '(':
                        tokens.append('(')
                        j = i + 1
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            j += 1
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j])
                        tokens.extend(['(', *inside_tokens, ')', '*', '-1', ')'])
                        i = j + 1
                    else:
                        print("Error")
                        break
            elif self._isvariable(exp[i]):
                var_start = i
                while i < len(exp) and self._isvariable(exp[i]):
                    i += 1
                tokens.append(exp[var_start:i])
                continue  # Ensure we skip the next else block after accumulating a variable
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(',expression.replace(" ", ""),')'))
        # print(tokenize_inner(exp)) #debug
        return self._tokenize_inner(exp)

In [77]:
expression = "a=(-lemon+2)"
print(''.join(('(',expression.replace(" ", ""),')')))

(a=(-lemon+2))


In [78]:
tkn = ExpressionTokenizer()

tkn.tokenize("a=(-lemon+2)")

['(', 'a', '=', '(', '(', 'l', '*', '-1', ')', 'emon', '+', '2', ')', ')']

In [93]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        # define criteria for variable
        return char.isalpha()

    def _isoperator(self, char):
        # define criteria for operator
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif self._isvariable(exp[i]):
                variable_name = ''
                while i < len(exp) and self._isvariable(exp[i]):
                    variable_name += exp[i]
                    i += 1
                tokens.append(variable_name)
            elif exp[i] == '-':
                if (i != 0 and (exp[i - 1]).isdigit() or self._isvariable(exp[i - 1]) or exp[i - 1] == ')') and ((exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]) or self._isoperator(exp[i + 1])):
                    tokens.append(exp[i])
                    i += 1
                else:
                    if self._isvariable(exp[i + 1]):
                        variable_name = ''
                        while i + 1 < len(exp) and self._isvariable(exp[i + 1]):
                            variable_name += exp[i + 1]
                            i += 1
                        tokens.extend(['(', variable_name, '*', '-1', ')'])
                        i += 1
                    elif (exp[i + 1]).isdigit() or self._isvariable(exp[i + 1]):
                        tokens.extend(['(', exp[i + 1], '*', '-1', ')'])
                        i += 2
                    elif exp[i + 1] == '(':
                        tokens.append('(')
                        j = i + 1
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            j += 1
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j])
                        tokens.extend(['(', *inside_tokens, ')', '*', '-1', ')'])
                        i = j + 1
                    else:
                        print("Error")
                        break
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(', expression.replace(" ", ""), ')'))
        return self._tokenize_inner(exp)
    
tkn = ExpressionTokenizer()
tokens = tkn.tokenize("-(lemon**20)")
print(tokens)

['(', '(', '(', 'lemon', '**', '2', '0', ')', '*', '-1', ')', ')']


In [105]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        return char.isalpha()

    def _isoperator(self, char):
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif self._isvariable(exp[i]):
                variable_name = exp[i]
                i += 1
                while i < len(exp) and self._isvariable(exp[i]):
                    variable_name += exp[i]
                    i += 1
                tokens.append(variable_name)
            elif exp[i].isdigit():
                number = exp[i]
                i += 1
                while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                    number += exp[i]
                    i += 1
                tokens.append(number)
            elif exp[i] == '-':
                if i + 1 < len(exp) and (exp[i + 1].isdigit() or self._isvariable(exp[i + 1]) or exp[i + 1] == '('):
                    if exp[i + 1] == '(':
                        j = i + 2
                        bracket_count = 1
                        while j < len(exp) and bracket_count > 0:
                            if exp[j] == '(':
                                bracket_count += 1
                            elif exp[j] == ')':
                                bracket_count -= 1
                            j += 1
                        inside_tokens = self._tokenize_inner(exp[i + 2:j-1])
                        tokens.extend(['(', *inside_tokens, '*', '-1', ')'])  # Correct placement of ')'
                        i = j  # Adjust to move past the processed section
                    elif exp[i + 1].isdigit():  # Negative number
                        number = exp[i + 1]
                        i += 2
                        while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                            number += exp[i]
                            i += 1
                        tokens.extend(['(', number, '*', '-1', ')'])
                    else:  # Negative variable
                        variable_name = exp[i + 1]
                        i += 2
                        while i < len(exp) and self._isvariable(exp[i]):
                            variable_name += exp[i]
                            i += 1
                        tokens.extend(['(', variable_name, '*', '-1', ')'])
                else:
                    tokens.append(exp[i])
                    i += 1
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(', expression.replace(" ", ""), ')'))
        return self._tokenize_inner(exp)

tkn = ExpressionTokenizer()
tokens = tkn.tokenize("-(-lemon**-20)")
print(tokens)


['(', '(', '(', 'lemon', '*', '-1', ')', '**', '(', '20', '*', '-1', ')', '*', '-1', ')', ')']


In [100]:
class ExpressionTokenizer:
    def __init__(self):
        pass

    def _isvariable(self, char):
        return char.isalpha()

    def _isoperator(self, char):
        return char in {'+', '-', '*', '/', '**'}

    def _tokenize_inner(self, exp):
        tokens = []
        i = 0
        while i < len(exp):
            if exp[i] == '*' and i + 1 < len(exp) and exp[i + 1] == '*':
                tokens.append('**')
                i += 2
            elif self._isvariable(exp[i]):
                variable_name = exp[i]
                i += 1
                while i < len(exp) and self._isvariable(exp[i]):
                    variable_name += exp[i]
                    i += 1
                tokens.append(variable_name)
            elif exp[i].isdigit():
                number = exp[i]
                i += 1
                while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                    number += exp[i]
                    i += 1
                tokens.append(number)
            elif exp[i] == '-':
                if i + 1 < len(exp) and (exp[i + 1].isdigit() or self._isvariable(exp[i + 1])):
                    # Handle negative numbers and variables specifically
                    i += 1
                    if exp[i].isdigit():  # Negative number
                        number = exp[i]
                        i += 1
                        while i < len(exp) and (exp[i].isdigit() or exp[i] == '.'):
                            number += exp[i]
                            i += 1
                        tokens.extend(['(', number, '*', '-1', ')'])
                    else:  # Negative variable
                        variable_name = exp[i]
                        i += 1
                        while i < len(exp) and self._isvariable(exp[i]):
                            variable_name += exp[i]
                            i += 1
                        tokens.extend(['(', variable_name, '*', '-1', ')'])
                else:
                    tokens.append(exp[i])
                    i += 1
            else:
                tokens.append(exp[i])
                i += 1
        return tokens

    def tokenize(self, expression):
        exp = ''.join(('(', expression.replace(" ", ""), ')'))
        return self._tokenize_inner(exp)
    
tkn = ExpressionTokenizer()
tokens = tkn.tokenize("-(-lemon+-20)")
print(tokens)


['(', '-', '(', '(', 'lemon', '*', '-1', ')', '+', '(', '20', '*', '-1', ')', ')', ')']
